In [20]:
from asyncio import protocols
from datetime import datetime,timedelta,date
import pandas as pd
from pyhive import presto
import numpy as np
import os
import warnings
from scipy import stats
import datetime
from datetime import datetime
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
presto_host ='bi-trino.serving.data.production.internal' #'presto.processing.yoda.run'
presto_port = '80'
presto_conn = presto.connect(
    host=presto_host,
    port=presto_port,
    username='praveen.u@rapido.bike',
    protocol='http',
    catalog='hive',
)
pd.options.mode.chained_assignment = None

In [13]:
#first divide the captains into reactivate, active captains

In [14]:
active=pd.read_csv('active_captains.csv')
reactive=pd.read_csv('reactivated_captains.csv')
lost=pd.read_csv('lost_caps.csv')

In [15]:
active=active.rename(columns={'reactivated_captains':'captain_id'})
reactive=reactive.rename(columns={'reactivated_captains':'captain_id'})

In [16]:
active['type']='active'
reactive['type']='reactive'
lost['type']='lost'

In [17]:
data=pd.concat([active,reactive,lost]).reset_index(drop=True)

In [18]:
data

,city,captain_id,type
0,Bangalore,6125f60634b93dc6e5311868,active
1,Bangalore,64085070244afdef8e487768,active
2,Bangalore,6256f243d1144e8638536698,active
3,Bangalore,642fccb4a8f49e807697b8d1,active
4,Bangalore,5d556f3295e84053d76491e4,active
...,...,...,...
101383,Hyderabad,612cdb7fb239c146a24289cb,lost
101384,Hyderabad,644236e38fac8a0c5dbba211,lost
101385,Hyderabad,61135738f7da3d26f5130bb0,lost
101386,Hyderabad,619895349273049b0195dc86,lost


In [ ]:
#as of 4th then as of 28th

In [25]:
query= """select city_name,captain_id, yyyymmdd, performance_segment, consistency,concat(performance_segment,'_',consistency) as pc_segment 
            from datasets_internal.captain_auto_pc_segments_v1 
            where yyyymmdd='20230428'
            and city_name in ('Bangalore','Chennai','Hyderabad')
            """
query_data=pd.read_sql(query,presto_conn)


In [26]:
query_data

,city_name,captain_id,yyyymmdd,performance_segment,consistency,pc_segment
0,Hyderabad,5ca0274b8c352421ea043ed3,20230428,LP,intra,LP_intra
1,Hyderabad,5d1975fa93686945e70c6873,20230428,MP,intra,MP_intra
2,Hyderabad,5d26e47f16609737fa46f15d,20230428,LP,inter,LP_inter
3,Hyderabad,5d2da3006ed7674e61145e12,20230428,LP,inter,LP_inter
4,Hyderabad,5d542bbc55fbf50d45f50e70,20230428,LP,inter,LP_inter
...,...,...,...,...,...,...
118429,Chennai,5ff6f59b857d1b3b38269a97,20230428,LP,inter,LP_inter
118430,Chennai,6007e2e9857d1becb22e1e53,20230428,LP,intra,LP_intra
118431,Chennai,60126d1ca5ef07048c88c559,20230428,LP,inter,LP_inter
118432,Chennai,6013e129027f252106a54f09,20230428,LP,inter,LP_inter


In [27]:
sub_data1=data.merge(query_data,on='captain_id',how='left')

In [24]:
# sub_data_v1=sub_data1.rename(columns={"yyyymmdd":'post_yyyymmdd','performance_segment':'pre_perf_seg',
#                          'consistency':'pre_consistent','pc_segment':'pre_segment'})

In [28]:
sub_data_v2=sub_data1.rename(columns={"yyyymmdd":'post_yyyymmdd','performance_segment':'post_perf_seg',
                         'consistency':'post_consistent','pc_segment':'post_segment'})

In [29]:
sub_data_v2

,city,captain_id,type,city_name,post_yyyymmdd,post_perf_seg,post_consistent,post_segment
0,Bangalore,6125f60634b93dc6e5311868,active,Bangalore,20230428,MP,inter,MP_inter
1,Bangalore,64085070244afdef8e487768,active,Bangalore,20230428,LP,inter,LP_inter
2,Bangalore,6256f243d1144e8638536698,active,Bangalore,20230428,LP,inter,LP_inter
3,Bangalore,642fccb4a8f49e807697b8d1,active,Bangalore,20230428,MP,intra,MP_intra
4,Bangalore,5d556f3295e84053d76491e4,active,Bangalore,20230428,LP,intra,LP_intra
...,...,...,...,...,...,...,...,...
101383,Hyderabad,612cdb7fb239c146a24289cb,lost,NaN,NaN,NaN,NaN,NaN
101384,Hyderabad,644236e38fac8a0c5dbba211,lost,NaN,NaN,NaN,NaN,NaN
101385,Hyderabad,61135738f7da3d26f5130bb0,lost,Hyderabad,20230428,HP,Daily,HP_Daily
101386,Hyderabad,619895349273049b0195dc86,lost,Hyderabad,20230428,MP,inter,MP_inter


In [31]:
#sub_data_v1=sub_data_v1.rename(columns={'post_yyyymmdd':'pre_yyyymmdd'})

In [32]:
fnal_merge=sub_data_v1[['city', 'captain_id', 'type','pre_yyyymmdd',
       'pre_perf_seg', 'pre_consistent', 'pre_segment']].merge(

sub_data_v2[['city', 'captain_id', 'type','post_yyyymmdd',
       'post_perf_seg', 'post_consistent', 'post_segment']]

,on= ['city','captain_id','type'],how='outer' )

In [194]:
fnal_merge.to_clipboard()

In [33]:
fnal_merge.head()

,city,captain_id,type,pre_yyyymmdd,pre_perf_seg,pre_consistent,pre_segment,post_yyyymmdd,post_perf_seg,post_consistent,post_segment
0,Bangalore,6125f60634b93dc6e5311868,active,20230407,MP,inter,MP_inter,20230428,MP,inter,MP_inter
1,Bangalore,64085070244afdef8e487768,active,20230407,MP,Daily,MP_Daily,20230428,LP,inter,LP_inter
2,Bangalore,6256f243d1144e8638536698,active,20230407,LP,inter,LP_inter,20230428,LP,inter,LP_inter
3,Bangalore,642fccb4a8f49e807697b8d1,active,NaN,NaN,NaN,NaN,20230428,MP,intra,MP_intra
4,Bangalore,5d556f3295e84053d76491e4,active,20230407,LP,intra,LP_intra,20230428,LP,intra,LP_intra


In [34]:
fnal_merge.type.unique()

array(['active', 'reactive', 'lost'], dtype=object)

In [35]:
fnal_merge=fnal_merge.drop_duplicates().reset_index(drop=True).fillna('NA')


In [39]:
fnal_merge.groupby(['city','type','pre_perf_seg','post_perf_seg'])[['captain_id']].nunique().to_clipboard()


In [37]:
fnal_merge[fnal_merge.type=='lost'].pre_segment.unique()

array(['LP_inter', 'LP_intra', 'NA', 'LP_Daily', 'HP_Daily', 'MP_intra',
       'MP_Daily', 'HP_inter', 'UHP_Daily', 'MP_inter', 'HP_intra',
       'UHP_intra', 'UHP_inter'], dtype=object)

### for comp intel based on seg movement

In [ ]:
### positive movement

In [202]:
p1=(fnal_merge[(fnal_merge.pre_perf_seg=='LP')&
            ((fnal_merge.post_perf_seg== 'MP')|
            (fnal_merge.post_perf_seg== 'HP')|
            (fnal_merge.post_perf_seg== 'UHP'))])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [203]:
p2=(fnal_merge[(fnal_merge.pre_perf_seg=='MP')&
            ((fnal_merge.post_perf_seg== 'HP')|
            (fnal_merge.post_perf_seg== 'UHP'))])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [204]:
p3=(fnal_merge[(fnal_merge.pre_perf_seg=='HP')&(fnal_merge.post_perf_seg== 'UHP')])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]



In [205]:
concat_positive=pd.concat([p1,p2,p3]).reset_index(drop=True)

In [206]:
concat_positive.head()

,city,type,captain_id,pre_perf_seg,post_perf_seg
0,Bangalore,active,64085070244afdef8e487768,LP,MP
1,Bangalore,active,601f52eaa6cc79712c21f9b2,LP,MP
2,Bangalore,active,64266f0b1dfee24e0b7d2d5b,LP,MP
3,Bangalore,active,5b801a7445271f10f571cd40,LP,MP
4,Bangalore,active,644cfce80ffc1693ff199ec7,LP,MP


In [207]:
### negative movement

In [208]:
n1=(fnal_merge[(fnal_merge.pre_perf_seg=='UHP')&
            ((fnal_merge.post_perf_seg== 'MP')|
            (fnal_merge.post_perf_seg== 'HP')|
            (fnal_merge.post_perf_seg== 'LP'))])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [209]:
n2=(fnal_merge[(fnal_merge.pre_perf_seg=='HP')&
            ((fnal_merge.post_perf_seg== 'MP')|
            (fnal_merge.post_perf_seg== 'LP'))])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [210]:
n3=(fnal_merge[(fnal_merge.pre_perf_seg=='MP')&
            (fnal_merge.post_perf_seg== 'LP')])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [211]:
negative_seg=pd.concat([n1,n2,n3]).reset_index(drop=True)

In [212]:
negative_seg.head()

,city,type,captain_id,pre_perf_seg,post_perf_seg
0,Bangalore,active,5c2f65114a267149c765a175,UHP,MP
1,Bangalore,active,618bac40fbd171a20ccc657c,UHP,HP
2,Bangalore,active,5bde68acb3d0e40f2857c3ca,UHP,HP
3,Bangalore,active,6292cc1f0b92512f62bcb615,UHP,HP
4,Bangalore,active,6113a339fefec89625aab507,UHP,LP


In [213]:
### no movement

In [214]:
nm1=(fnal_merge[(fnal_merge.pre_perf_seg=='UHP')&
            (fnal_merge.post_perf_seg== 'UHP')])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [215]:
nm2=(fnal_merge[(fnal_merge.pre_perf_seg=='HP')&
            (fnal_merge.post_perf_seg== 'HP')])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [216]:
nm3=(fnal_merge[(fnal_merge.pre_perf_seg=='LP')&
            (fnal_merge.post_perf_seg== 'LP')])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [217]:
nm4=(fnal_merge[(fnal_merge.pre_perf_seg=='MP')&
            (fnal_merge.post_perf_seg== 'MP')])[['city','type','captain_id','pre_perf_seg','post_perf_seg']]

In [218]:
no_move_Data=pd.concat([nm1,nm2,nm3,nm4]).reset_index(drop=True)

In [219]:
no_move_Data.head()

,city,type,captain_id,pre_perf_seg,post_perf_seg
0,Bangalore,active,6270bdb49848f8b1fd8244ee,UHP,UHP
1,Bangalore,active,60e31e1d09aa9966ba407ace,UHP,UHP
2,Bangalore,active,5d1ce1693b752c45cf957614,UHP,UHP
3,Bangalore,active,60e123ab569b2607fb7067e1,UHP,UHP
4,Bangalore,active,5ae44459f8ece007309fc9e9,UHP,UHP


In [220]:
#

In [221]:
## getting comp intel data

In [292]:
che=pd.read_csv('che_comp_intel_week_wise.csv')
blr=pd.read_csv('blr_comp_intel_week_wise.csv')

In [293]:
# che=pd.read_csv('compl_intel_chennai_whole_city.csv')
# blr=pd.read_csv('comp_intel_bangalore_whole_city.csv')

In [294]:
blr_che=pd.concat([blr,che])

In [295]:
blr_che

,city,captain_id,week,Post_Comp_per,Post_Ola_per,Post_Uber_per,Post_Namma_per,Post_Indrive_per
0,Bangalore,646b048500e39c538e1ba10b,21,0.00,0.00,0.0,0.00,0.0
1,Bangalore,646b048500e39c538e1ba10b,20,0.00,0.00,0.0,0.00,0.0
2,Bangalore,62be88c2bd09423d98a6edab,20,31.94,31.94,0.0,0.00,0.0
3,Bangalore,62be88c2bd09423d98a6edab,21,44.29,44.29,0.0,9.76,0.0
4,Bangalore,62be88c2bd09423d98a6edab,19,28.57,28.57,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...
46977,Chennai,5d561298bde10e0d54408d2f,21,0.00,0.00,0.0,0.00,0.0
46978,Chennai,629477ae98c750e77357b3e1,20,0.00,0.00,0.0,0.00,0.0
46979,Chennai,6194c9724be12d0697bb31fa,20,0.00,0.00,0.0,0.00,0.0
46980,Chennai,6194c9724be12d0697bb31fa,19,0.00,0.00,0.0,0.00,0.0


In [224]:
hyd=pd.read_csv('comp_intel_hyd_whole_city.csv')

In [ ]:
concat_positive
negative_seg
no_move_Data

In [290]:
concat_positive

,city,type,captain_id,pre_perf_seg,post_perf_seg
0,Bangalore,active,64085070244afdef8e487768,LP,MP
1,Bangalore,active,601f52eaa6cc79712c21f9b2,LP,MP
2,Bangalore,active,64266f0b1dfee24e0b7d2d5b,LP,MP
3,Bangalore,active,5b801a7445271f10f571cd40,LP,MP
4,Bangalore,active,644cfce80ffc1693ff199ec7,LP,MP
...,...,...,...,...,...
13970,Hyderabad,lost,6273949bef22ea85293f6f18,HP,UHP
13971,Hyderabad,lost,60237df7973b0fc4f63c9e30,HP,UHP
13972,Hyderabad,lost,5c46ed8d4a267149c7777f9b,HP,UHP
13973,Hyderabad,lost,5df52374be73542b52df02e0,HP,UHP


In [ ]:
#positive blr & che

In [296]:
pos_comp=(concat_positive[(concat_positive.city=='Bangalore')|(concat_positive.city=='Chennai')]
.merge(blr_che[['captain_id','week', 'Post_Comp_per',
       'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per', 'Post_Indrive_per']],on=['captain_id']))

In [297]:
pos_comp['movement']='positive'

In [311]:
pos_comp.groupby(['city','movement','week','type'],as_index=False)[['captain_id']].count().to_clipboard()

In [299]:
pos_comp.groupby(['city','movement','type','week'])[['Post_Comp_per', 'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per','Post_Indrive_per']].mean()



Post_Comp_per  Post_Ola_per  Post_Uber_per  \
city      movement type     week                                               
Bangalore positive active   19        31.140147     18.392256      10.157222   
                            20        32.266932     17.076962      10.970098   
                            21        33.690256     17.896020      10.981856   
                   lost     19        45.948000     29.208000      19.260000   
                            20        31.552727      7.340909      14.834545   
                            21        32.627000     10.928000      17.219000   
                   reactive 19        34.647973     18.956554      16.709595   
                            20        35.631082     19.625155      17.595773   
                            21        36.083286     19.692476      18.306667   
Chennai   positive active   19        34.026467     23.504408      15.850592   
                            20        33.558655     22.366643      15.995625   
                            21        33.825351     23.071833      14.995333   
                   lost     19        45.360000     43.990000      12.620000   
                            20        35.850000     35.584286      10.055714   
                            21        36.311250     36.383750       6.406250   
                   reactive 19        34.196643     23.926294      15.948322   
                            20        31.967412     23.074294      14.350412   
                            21        33.771732     23.770559      15.704581   

                                  Post_Namma_per  Post_Indrive_per  
city      movement type     week                                    
Bangalore positive active   19         15.640509          0.038748  
                            20         16.826222          0.020373  
                            21         17.311701          0.026437  
                   lost     19         21.465000          0.000000  
                            20         14.541818          0.000000  
                            21         17.748000          0.000000  
                   reactive 19          9.109189          0.000000  
                            20         10.508711          0.000000  
                            21          9.948238          0.000000  
Chennai   positive active   19          0.029518          8.816563  
                            20          0.104523         10.385695  
                            21          0.111591         10.359975  
                   lost     19          0.000000          0.000000  
                            20          0.000000          0.985714  
                            21          0.000000          0.000000  
                   reactive 19          0.000000          5.329860  
                            20          0.000000          8.029235  
                            21          0.000000          6.801117

In [300]:
pos_comp.groupby(['city','movement','week','type'])[['Post_Comp_per', 'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per','Post_Indrive_per']].mean().to_clipboard()



In [ ]:
#negative

In [301]:
neg_comp=(negative_seg[(negative_seg.city=='Bangalore')|(negative_seg.city=='Chennai')]
.merge(blr_che[['captain_id','week' ,'Post_Comp_per',
       'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per', 'Post_Indrive_per']],on=['captain_id']))

In [302]:
neg_comp['movement']='negative'

In [312]:
neg_comp.groupby(['city','movement','week','type'],as_index=False)[['captain_id']].count().to_clipboard()

In [304]:
neg_comp.groupby(['city','movement','type','week'])[['Post_Comp_per', 'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per','Post_Indrive_per']].mean()



Post_Comp_per  Post_Ola_per  Post_Uber_per  \
city      movement type     week                                               
Bangalore negative active   19        36.172233     21.775965      11.155446   
                            20        38.773135     21.574791      12.430775   
                            21        38.933482     21.208485      12.485295   
                            22       100.000000    100.000000       0.000000   
                   lost     19        52.808158     22.191053      26.961316   
                            20        37.743111     12.946889      24.616222   
                            21        39.754255     15.401064      18.009787   
                   reactive 19        36.825855     23.029671      16.640000   
                            20        41.418389     21.187444      24.191389   
                            21        40.710865     19.357297      21.545189   
Chennai   negative active   19        40.157913     30.256677      16.481890   
                            20        39.332417     28.483880      17.285312   
                            21        39.581687     28.748981      16.967091   
                   lost     19        27.514000     20.048000      10.566000   
                            20        26.458571     14.738571      12.851905   
                            21        38.505500     30.526000       8.409500   
                   reactive 19        39.610094     29.188491      16.330377   
                            20        40.473438     28.455078      16.289922   
                            21        38.554567     25.005512      19.726142   

                                  Post_Namma_per  Post_Indrive_per  
city      movement type     week                                    
Bangalore negative active   19         18.682272          0.058766  
                            20         19.186385          0.024016  
                            21         20.091617          0.051480  
                            22          0.000000          0.000000  
                   lost     19         18.136053          0.000000  
                            20          4.872000          1.111111  
                            21         15.145957          0.000000  
                   reactive 19         10.452829          0.000000  
                            20          9.983833          0.000000  
                            21         10.242270          0.000000  
Chennai   negative active   19          0.000000          7.502520  
                            20          0.000000          8.360845  
                            21          0.000000          8.155562  
                   lost     19          0.000000          0.000000  
                            20          0.000000          5.578571  
                            21          0.000000          2.272500  
                   reactive 19          0.000000          9.855000  
                            20          0.000000          7.355547  
                            21          0.000000          5.387717

In [305]:
neg_comp.groupby(['city','movement','type','week'])[['Post_Comp_per', 'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per','Post_Indrive_per']].mean().to_clipboard()



In [ ]:
#no movement

In [306]:
no_comp=(no_move_Data[(no_move_Data.city=='Bangalore')|(no_move_Data.city=='Chennai')]
.merge(blr_che[['captain_id','week' ,'Post_Comp_per',
       'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per', 'Post_Indrive_per']],on=['captain_id']))

In [307]:
no_comp['movement']='no_movement'

In [313]:
no_comp.groupby(['city','movement','week','type'],as_index=False)[['captain_id']].count().to_clipboard()

In [309]:
no_comp.groupby(['city','movement','type','week'])[['Post_Comp_per', 'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per','Post_Indrive_per']].mean()



Post_Comp_per  Post_Ola_per  \
city      movement    type     week                                
Bangalore no_movement active   19        34.089138     24.064788   
                               20        36.263263     24.427891   
                               21        36.648349     24.237827   
                      lost     19        33.910493     23.621056   
                               20        36.994881     23.505862   
                               21        36.940000     23.381447   
                      reactive 19        34.806795     25.495049   
                               20        36.874691     25.486508   
                               21        38.196262     25.688548   
Chennai   no_movement active   19        34.204572     25.259072   
                               20        33.950966     24.589054   
                               21        34.806258     25.687951   
                      lost     19        28.640376     21.183910   
                               20        24.676225     21.179272   
                               21        28.774631     20.680201   
                      reactive 19        35.401650     27.319612   
                               20        34.547601     25.719470   
                               21        35.108688     26.779233   

                                     Post_Uber_per  Post_Namma_per  \
city      movement    type     week                                  
Bangalore no_movement active   19        12.339716       18.890452   
                               20        12.898307       19.549898   
                               21        13.260639       20.000423   
                      lost     19        16.174507       13.887887   
                               20        18.154271       13.727241   
                               21        17.496737       13.607184   
                      reactive 19        15.867400       16.088435   
                               20        16.926645       16.277492   
                               21        16.762023       17.729917   
Chennai   no_movement active   19        16.095616        0.034999   
                               20        16.509960        0.027843   
                               21        16.413710        0.026566   
                      lost     19        12.728496        0.000000   
                               20         8.899934        0.000000   
                               21        12.476309        0.000000   
                      reactive 19        17.454327        0.127809   
                               20        18.298207        0.038851   
                               21        17.748354        0.000000   

                                     Post_Indrive_per  
city      movement    type     week                    
Bangalore no_movement active   19            0.012043  
                               20            0.026194  
                               21            0.021889  
                      lost     19            0.000000  
                               20            0.000000  
                               21            0.000000  
                      reactive 19            0.000000  
                               20            0.000000  
                               21            0.000000  
Chennai   no_movement active   19            7.817321  
                               20            8.130224  
                               21            8.273602  
                      lost     19            2.986917  
                               20            3.961987  
                               21            4.243691  
                      reactive 19            6.709057  
                               20            7.247260  
                               21            6.877401

In [310]:
no_comp.groupby(['city','movement','type','week'])[['Post_Comp_per', 'Post_Ola_per', 'Post_Uber_per', 'Post_Namma_per','Post_Indrive_per']].mean().to_clipboard()



In [ ]:
#HYD

In [266]:
#positive
hyd_pst=(concat_positive[(concat_positive.city=='Hyderabad')]
.merge(hyd[['captain_id', 'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per']],on=['captain_id']))

In [268]:
hyd_pst['movement']='positive'

In [269]:
hyd_pst.groupby(['city','movement','type'])[['captain_id']].count()

captain_id
city      movement type                
Hyderabad positive active          4391
                   lost              27
                   reactive         351

In [270]:
hyd_pst.head(2)

,city,type,captain_id,pre_perf_seg,post_perf_seg,Pre_Comp_per,Post_Comp_per,Pre_Ola_per,Post_Ola_per,Pre_Uber_per,Post_Uber_per,Pre_Namma_per,Post_Namma_per,Pre_Indrive_per,Post_Indrive_per,movement
0,Hyderabad,active,5f4744a122bbb3bced006f77,LP,MP,46.48,48.11,0.0,30.65,71.74,32.89,0.0,0.0,0.0,0.0,positive
1,Hyderabad,active,61b4423cf3de6f0b774949e0,LP,HP,0.00,0.70,0.0,0.79,0.00,0.00,0.0,0.0,0.0,0.0,positive


In [271]:
hyd_pst.groupby(['city','movement','type'])[[
    'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per'
]].mean()



Pre_Comp_per  Post_Comp_per  Pre_Ola_per  \
city      movement type                                                 
Hyderabad positive active       36.143220      28.215192    22.428636   
                   lost         29.775926      25.374815    21.293704   
                   reactive     38.657835      32.992735    23.270114   

                             Post_Ola_per  Pre_Uber_per  Post_Uber_per  \
city      movement type                                                  
Hyderabad positive active       17.980726     18.941760      16.071462   
                   lost         12.738148     13.667778      17.459259   
                   reactive     19.629858     21.187123      20.647379   

                             Pre_Namma_per  Post_Namma_per  Pre_Indrive_per  \
city      movement type                                                       
Hyderabad positive active              0.0        0.043796              0.0   
                   lost                0.0        0.000000              0.0   
                   reactive            0.0        0.268148              0.0   

                             Post_Indrive_per  
city      movement type                        
Hyderabad positive active            0.210251  
                   lost              0.000000  
                   reactive          0.342821

In [287]:
hyd_pst.groupby(['city','movement','type'])[[
    'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per'
]].mean().to_clipboard()

In [275]:
#negative

hyd_neg=(negative_seg[(negative_seg.city=='Hyderabad')]
.merge(hyd[['captain_id', 'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per']],on=['captain_id']))

In [276]:
hyd_neg['movement']='negative'

In [277]:
hyd_neg.groupby(['city','movement','type'])[['captain_id']].count()

captain_id
city      movement type                
Hyderabad negative active          1483
                   lost              44
                   reactive         119

In [278]:
hyd_neg.groupby(['city','movement','type'])[[
    'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per'
]].mean()



Pre_Comp_per  Post_Comp_per  Pre_Ola_per  \
city      movement type                                                 
Hyderabad negative active       31.366339      37.090472    18.446460   
                   lost         31.110227      31.051364    15.587955   
                   reactive     44.773866      48.674370    22.188235   

                             Post_Ola_per  Pre_Uber_per  Post_Uber_per  \
city      movement type                                                  
Hyderabad negative active       21.885543     17.080061      20.926399   
                   lost         14.579318     21.502955      19.470455   
                   reactive     27.320000     27.933193      30.328992   

                             Pre_Namma_per  Post_Namma_per  Pre_Indrive_per  \
city      movement type                                                       
Hyderabad negative active              0.0        0.033203              0.0   
                   lost                0.0        0.000000              0.0   
                   reactive            0.0        0.000000              0.0   

                             Post_Indrive_per  
city      movement type                        
Hyderabad negative active            0.303891  
                   lost              0.000000  
                   reactive          0.070000

In [288]:
hyd_neg.groupby(['city','movement','type'])[[
    'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per'
]].mean().to_clipboard()



In [ ]:
#no movement

In [280]:
#no movement

hyd_no=(no_move_Data[(no_move_Data.city=='Hyderabad')]
.merge(hyd[['captain_id', 'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per']],on=['captain_id']))

In [281]:
hyd_no['movement']='no_movement'

In [282]:
hyd_no.groupby(['city','movement','type'])[['captain_id']].count()

captain_id
city      movement    type                
Hyderabad no_movement active         12114
                      lost             285
                      reactive        1189

In [283]:
hyd_no.groupby(['city','movement','type'])[[
    'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per'
]].mean()



Pre_Comp_per  Post_Comp_per  Pre_Ola_per  \
city      movement    type                                                 
Hyderabad no_movement active       33.633032      33.897334    23.959871   
                      lost         36.231789      32.916211    24.999719   
                      reactive     41.269899      43.942506    29.839243   

                                Post_Ola_per  Pre_Uber_per  Post_Uber_per  \
city      movement    type                                                  
Hyderabad no_movement active       24.628224     15.628176      16.690926   
                      lost         22.982246     16.736912      15.188070   
                      reactive     32.042733     19.234643      21.731842   

                                Pre_Namma_per  Post_Namma_per  \
city      movement    type                                      
Hyderabad no_movement active              0.0        0.015324   
                      lost                0.0        0.000000   
                      reactive            0.0        0.000000   

                                Pre_Indrive_per  Post_Indrive_per  
city      movement    type                                         
Hyderabad no_movement active                0.0          0.143025  
                      lost                  0.0          0.350877  
                      reactive              0.0          0.338259

In [289]:
hyd_no.groupby(['city','movement','type'])[[
    'Pre_Comp_per',
       'Post_Comp_per', 'Pre_Ola_per', 'Post_Ola_per', 'Pre_Uber_per',
       'Post_Uber_per', 'Pre_Namma_per', 'Post_Namma_per', 'Pre_Indrive_per',
       'Post_Indrive_per'
]].mean().to_clipboard()

